<a href="https://colab.research.google.com/github/joevimaljohn/NASA-_NEO_TRACKING_INSIGHTS/blob/main/nasa_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
API_KEY="lUSA1klJSHHXeBfelE8Co3TT0DIfwqlm9EOgGFMO"
import requests

url  = f"https://api.nasa.gov/neo/rest/v1/feed?start_date=2024-01-01&end_date=2024-01-07&api_key={API_KEY}"

response = requests.get(url)

response

In [ ]:
data = response.json()

data

In [ ]:
data.keys()

In [ ]:
data['links']['next']

In [ ]:
data['element_count']

In [ ]:

approach_date = data['near_earth_objects']['2024-01-01'][0]['close_approach_data'][0]['close_approach_date']
approach_date_obj = datetime.strptime(approach_date, "%Y-%m-%d")
approach_date_obj

In [ ]:
data['near_earth_objects']['2024-01-01'][0]['estimated_diameter']['kilometers']['estimated_diameter_min']

In [ ]:
data['near_earth_objects'].keys()

In [ ]:
!pip install streamlit

In [ ]:
!pip install mysql.connector

In [29]:
import requests
from datetime import datetime



API_KEY = "lUSA1klJSHHXeBfelE8Co3TT0DIfwqlm9EOgGFMO"
asteroids_data = []
target = 10000
url = f"https://api.nasa.gov/neo/rest/v1/feed?start_date=2024-01-01&end_date=2024-01-07&api_key={API_KEY}"

while len(asteroids_data) < target:
    response = requests.get(url)
    data = response.json()
    details = data['near_earth_objects']

    for date, asteroids in details.items():
        for ast in asteroids:

            if not ast['close_approach_data']:
                continue

            approach = ast['close_approach_data'][0]

            approach_date = approach['close_approach_date']
            approach_date_obj = datetime.strptime(approach_date, "%Y-%m-%d")


            asteroid_info = dict(
                id=int(ast['id']),
                neo_ref_id=int(ast['neo_reference_id']),
                name=ast['name'],
                magnitude=float(ast['absolute_magnitude_h']),
                estimated_dia_min=float(ast['estimated_diameter']['kilometers']['estimated_diameter_min']),
                estimated_dia_max=float(ast['estimated_diameter']['kilometers']['estimated_diameter_max']),
                hazardous_asteroid=bool(ast['is_potentially_hazardous_asteroid']),
                close_approach_date=approach_date_obj,
                relativevelocity=float(approach['relative_velocity']['kilometers_per_hour']),
                astronomical=float(approach['miss_distance']['astronomical']),
                missdistancekm=float(approach['miss_distance']['kilometers']),
                missdistancelunar=float(approach['miss_distance']['lunar']),
                orbitingbody=approach['orbiting_body']
            )

            asteroids_data.append(asteroid_info)

            if len(asteroids_data) >= target:
                break
        if len(asteroids_data) >= target:
            break


    url = data.get('links', {}).get('next', None)
    if not url:
        break



In [30]:
len(asteroids_data)

10000

In [31]:
asteroids_data[0]

{'id': 2415949,
 'neo_ref_id': 2415949,
 'name': '415949 (2001 XY10)',
 'magnitude': 19.37,
 'estimated_dia_min': 0.3552670883,
 'estimated_dia_max': 0.7944013596,
 'hazardous_asteroid': False,
 'close_approach_date': datetime.datetime(2024, 1, 2, 0, 0),
 'relativevelocity': 57205.8951204341,
 'astronomical': 0.3372535274,
 'missdistancekm': 50452409.349026635,
 'missdistancelunar': 131.1916221586,
 'orbitingbody': 'Earth'}

In [27]:
!pip install mysql.connector

In [23]:
!pip install mysql-connector-python

In [28]:
import mysql.connector

# Establish the connection
conn = mysql.connector.connect(
    host="gateway01.ap-southeast-1.prod.aws.tidbcloud.com",  # or your actual host/IP
    user="2ZoP6qND9CYzFo1.root",  # your MySQL username
    password="JL7wxYjzWssXnkdm",   # your MySQL password
    port=4000,                     # custom port, fine if your server uses it

   )
cursor = conn.cursor()

cursor.execute("CREATE DATABASE IF NOT EXISTS NASA")
cursor.execute("USE NASA")

# Create asteroids table
table_query_as = """
CREATE TABLE IF NOT EXISTS asteroids(
    id INT,
    name VARCHAR(255),
    magnitude FLOAT,
    estimated_diameter_min FLOAT,
    estimated_diameter_max FLOAT,
    is_potentially_hazardous_asteroid  BOOLEAN
)
"""

# Create close_approach table
table_query_cl = """
CREATE TABLE IF NOT EXISTS close_approach(
    neo_reference_id INT,
    close_approach_date DATE,
    relative_velocity_kmph FLOAT,
    astronomical_AU FLOAT,
    miss_distance_km FLOAT,
    miss_distance_lunar FLOAT,
    orbiting_body VARCHAR(255)
)
"""

cursor.execute(table_query_as)
cursor.execute(table_query_cl)

insert_query = """
INSERT INTO asteroids (id, name, magnitude, estimated_diameter_min, estimated_diameter_max, is_potentially_hazardous_asteroid)
VALUES (%s, %s, %s, %s, %s, %s)
"""
insert_close_approach_query = """
INSERT INTO close_approach (
    neo_reference_id,
    close_approach_date,
    relative_velocity_kmph,
    astronomical_AU,
    miss_distance_km,
    miss_distance_lunar,
    orbiting_body
)
VALUES (%s, %s, %s, %s, %s, %s, %s)
"""


In [32]:

asteroid_rows = []
close_approach_rows = []


for asteroid in asteroids_data:
    asteroid_row = (
        asteroid['id'],
        asteroid['name'],
        asteroid['magnitude'],
        asteroid['estimated_dia_min'],
        asteroid['estimated_dia_max'],
        asteroid['hazardous_asteroid']
    )
    asteroid_rows.append(asteroid_row)

for asteroid in asteroids_data:
  close_approach_row = (
            asteroid['neo_ref_id'],
            asteroid['close_approach_date'],
            asteroid['relativevelocity'],
            asteroid['astronomical'],
            asteroid['missdistancekm'],
            asteroid['missdistancelunar'],
            asteroid['orbitingbody']
        )
  close_approach_rows.append(close_approach_row)

cursor.executemany(insert_query, asteroid_rows)
cursor.executemany(insert_close_approach_query, close_approach_rows)


conn.commit()

Find the average miss distance (in km) for each asteroid,Include the number of approaches per **asteroid**

In [ ]:
cursor.execute("""
SELECT
    neo_reference_id,
    COUNT(*) AS approach_count,
    AVG(miss_distance_km) AS avg_miss_distance_km
FROM
    close_approach
GROUP BY
    neo_reference_id;

""")
rows = cursor.fetchall()
for row in rows:
    print(f"Asteroid ID: {row[0]}, Approaches: {row[1]}, Avg Miss Distance (km): {row[2]:,.2f}")

Asteroid with fastest ever approach speed

In [ ]:
cursor.execute("""
SELECT *
FROM close_approach
ORDER BY relative_velocity_kmph DESC
LIMIT 1;
"""
)
row = cursor.fetchone()

# Print the result in a readable format
columns = [desc[0] for desc in cursor.description]  # Get column names
result = dict(zip(columns, row))

for key, value in result.items():
    print(f"{key}: {value}")


neo_reference_id: 54392072
close_approach_date: 2024-11-22
relative_velocity_kmph: 173071.83
astronomical_AU: 0.49434707
miss_distance_km: 73953260.0
miss_distance_lunar: 192.30101
orbiting_body: Earth


Identify the asteroids that approached earch on weekend

In [ ]:

cursor.execute("""
SELECT *
FROM close_approach
WHERE DAYOFWEEK(close_approach_date) IN (1, 7);
"""
)
rows = cursor.fetchall()

columns = [desc[0] for desc in cursor.description]

for row in rows:
    record = dict(zip(columns, row))
    print(record)


Find asteroids that are unusually dim (magnitude <20 )

In [ ]:
cursor.execute("""
SELECT name, magnitude
FROM asteroids
WHERE magnitude < 20;
"""
)
rows = cursor.fetchall()

# Get column names
columns = [desc[0] for desc in cursor.description]

# Print results
for row in rows:
    record = dict(zip(columns, row))
    print(record)


List all asteroids that approached Earth with a relative velocity more than 2 times the average velocity of all approaches

In [ ]:
query = """
SELECT *
FROM close_approach
WHERE relative_velocity_kmph > (
    SELECT 2 * AVG(relative_velocity_kmph) FROM close_approach
);
"""

cursor.execute(query)
rows = cursor.fetchall()

# Display results with column names
columns = [desc[0] for desc in cursor.description]
for row in rows:
    print(dict(zip(columns, row)))


In [37]:
cursor.execute("""
SELECT
    id,
    name,
    COUNT(*) AS approach_count
FROM
    asteroids
WHERE  is_potentially_hazardous_asteroid = true

GROUP BY
    id, name
HAVING
    COUNT(*) > 3
ORDER BY
    approach_count DESC
""")

# Fetch and print results
results = cursor.fetchall()

# Display neatly
print("Hazardous Asteroids with More Than 3 Approaches:\n")
print("{:<6} {:<15} {:<5}".format("ID", "Name", "Count"))
print("-" * 30)
for row in results:
    print("{:<6} {:<15} {:<5}".format(row[0], row[1], row[2]))

Hazardous Asteroids with More Than 3 Approaches:

ID     Name            Count
------------------------------
54378880 (2023 QL7)      4    
3591722 (2011 YJ28)     4    


In [ ]:
query = """
SELECT id, name, estimated_diameter_min, estimated_diameter_max,
       (estimated_diameter_max - estimated_diameter_min) AS diameter_range
FROM asteroids
WHERE (estimated_diameter_max - estimated_diameter_min) > 0.3
ORDER BY diameter_range DESC
"""

cursor.execute(query)
rows = cursor.fetchall()

# Get column names
columns = [desc[0] for desc in cursor.description]

# Convert to dictionary list (or DataFrame)
results = [dict(zip(columns, row)) for row in rows]

# Print or display
for asteroid in results:
    print(asteroid)


In [41]:
%%writefile app.py
import streamlit as st
import pandas as pd
import mysql.connector
import plotly.express as px

# Connect to DB
@st.cache_resource
def connect_db():
    return mysql.connector.connect(
        host="gateway01.ap-southeast-1.prod.aws.tidbcloud.com",
        user="2ZoP6qND9CYzFo1.root",  # your MySQL username
        password="JL7wxYjzWssXnkdm",
        port=4000,
        database="NASA"
    )

conn = connect_db()
cursor = conn.cursor(dictionary=True)

st.set_page_config(page_title="NASA Asteroids Dashboard", layout="wide")

# --- Sidebar ---
st.sidebar.title("🔭 NASA Dashboard Options")
selected_table = st.sidebar.selectbox("Choose Table", ["asteroids", "close_approach"])

@st.cache_data
def load_table(table_name):
    cursor.execute(f"SELECT * FROM {table_name}")
    return pd.DataFrame(cursor.fetchall())

df = load_table(selected_table)

# --- Metrics ---
st.title("☄️ NASA Asteroid & Close Approach Data Explorer")
col1, col2, col3 = st.columns(3)
col1.metric("📦 Total Records", len(df))
if selected_table == "asteroids":
    hazard_count = df['is_potentially_hazardous_asteroid'].value_counts().get("true", 0)
    col2.metric("⚠️ Hazardous", hazard_count)
    col3.metric("🪨 Avg. Diameter (km)", round(df['estimated_diameter_max'].mean(), 2))

# --- Tabs Layout ---
tab1, tab2, tab3, tab4 = st.tabs(["📋 Table", "🔍 Filters", "🧠 SQL Playground", "📊 Charts"])

# --- Tab 1: Raw Table View ---
with tab1:
    st.subheader(f"Raw Data: {selected_table}")
    st.dataframe(df, use_container_width=True)
    st.download_button("📥 Download CSV", df.to_csv(index=False), file_name=f"{selected_table}.csv")

# --- Tab 2: Filtered View ---
with tab2:
    st.subheader("🔍 Filter Controls")

    if selected_table == "asteroids":
        mag_range = st.slider("Magnitude Range", 0.0, 40.0, (15.0, 25.0))
        dia_range = st.slider("Estimated Diameter (min)", 0.0, 2.0, (0.1, 1.0))
        hazard_filter = st.checkbox("Only show hazardous asteroids")

        filtered_df = df[
            (df['magnitude'].between(*mag_range)) &
            (df['estimated_diameter_min'].between(*dia_range))
        ]
        if hazard_filter:
            filtered_df = filtered_df[df['is_potentially_hazardous_asteroid'] == "Yes"]

        st.dataframe(filtered_df, use_container_width=True)

    elif selected_table == "close_approach":
        st.subheader("📅 Filter Close Approaches by Date Range")

        # Ensure date column is datetime type
        df['close_approach_date'] = pd.to_datetime(df['close_approach_date'])

        min_date = df['close_approach_date'].min().date()
        max_date = df['close_approach_date'].max().date()

        date_range = st.date_input(
            "Select Close Approach Date Range",
            value=(min_date, max_date),
            min_value=min_date,
            max_value=max_date
        )

        if isinstance(date_range, tuple) and len(date_range) == 2:
            start_date, end_date = date_range
            filtered_df = df[
                (df['close_approach_date'] >= pd.to_datetime(start_date)) &
                (df['close_approach_date'] <= pd.to_datetime(end_date))
            ]
            st.success(f"✅ Showing records from {start_date} to {end_date}")
            st.dataframe(filtered_df, use_container_width=True)
        else:
            st.info("ℹ️ Please select a valid date range to view filtered results.")


# --- Tab 3: SQL Playground ---
with tab3:
    st.subheader("🧠 Predefined SQL Queries")

    predefined_queries = [
    {
        "title": "🧮 Count how many times each asteroid has approached Earth",
        "sql": """
        SELECT neo_reference_id, COUNT(*) AS approach_count
        FROM close_approach
        GROUP BY neo_reference_id
        ORDER BY approach_count DESC
        """
    },
    {
        "title": "🚀 Average velocity of each asteroid over multiple approaches",
        "sql": """
        SELECT neo_reference_id, AVG(relative_velocity_kmph) AS avg_velocity
        FROM close_approach
        GROUP BY neo_reference_id
        ORDER BY avg_velocity DESC
        """
    },
    {
        "title": "⚡ Top 10 fastest asteroids",
        "sql": """
        SELECT * FROM close_approach
        ORDER BY relative_velocity_kmph DESC
        LIMIT 10
        """
    },
    {
        "title": "☄️ Hazardous asteroids with more than 3 approaches",
        "sql": """SELECT
        id,name,
        COUNT(*) AS approach_count
        FROM
        asteroids
        WHERE  is_potentially_hazardous_asteroid = true
        GROUP BY
        id, name HAVING
        COUNT(*) > 3
        ORDER BY
        approach_count DESC
        """
    },
    {
        "title": "📅 Month with the most asteroid approaches",
        "sql": """
        SELECT MONTH(close_approach_date) AS month, COUNT(*) AS count
        FROM close_approach
        GROUP BY month
        ORDER BY count DESC
        LIMIT 1
        """
    },
    {
        "title": "🚀 Asteroid with fastest ever approach speed",
        "sql": """
        SELECT * FROM close_approach
        ORDER BY relative_velocity_kmph DESC
        LIMIT 1
        """
    },
    {
        "title": "📏 Sort asteroids by max estimated diameter (desc)",
        "sql": """
        SELECT * FROM asteroids
        ORDER BY estimated_diameter_max DESC
        LIMIT 10
        """
    },
    {
        "title": "📉 Asteroids getting closer over time",
        "sql": """
        SELECT neo_reference_id, close_approach_date, miss_distance_km
        FROM close_approach
        ORDER BY neo_reference_id, close_approach_date
        """
    },
    {
        "title": "📌 Closest approach for each asteroid (name, date, distance)",
        "sql": """
        SELECT a.name, ca.close_approach_date, ca.miss_distance_km
        FROM asteroids a
        JOIN close_approach ca ON a.id = ca.neo_reference_id
        WHERE (ca.neo_reference_id, ca.miss_distance_km) IN (
            SELECT neo_reference_id, MIN(miss_distance_km)
            FROM close_approach
            GROUP BY neo_reference_id
        )
        ORDER BY ca.miss_distance_km ASC
        """
    },
    {
        "title": "💨 Asteroids with velocity > 50,000 km/h",
        "sql": """
        SELECT * FROM close_approach
        WHERE relative_velocity_kmph > 50000
        ORDER BY relative_velocity_kmph DESC
        """
    },
    {
        "title": "📊 Count of approaches per month",
        "sql": """
        SELECT MONTH(close_approach_date) AS month, COUNT(*) AS total
        FROM close_approach
        GROUP BY month
        ORDER BY month
        """
    },
    {
        "title": "✨ Asteroid with highest brightness (lowest magnitude)",
        "sql": """
        SELECT * FROM asteroids
        ORDER BY magnitude ASC
        LIMIT 1
        """
    },
    {
        "title": "🧮 Hazardous vs Non-Hazardous asteroid counts",
        "sql": """
        SELECT is_potentially_hazardous_asteroid, COUNT(*) AS total
        FROM asteroids
        GROUP BY is_potentially_hazardous_asteroid
        """
    },
    {
        "title": "🌙 Asteroids that passed closer than the Moon (< 1 LD)",
        "sql": """
        SELECT a.name, ca.close_approach_date, ca.miss_distance_lunar
        FROM asteroids a
        JOIN close_approach ca ON a.id = ca.neo_reference_id
        WHERE ca.miss_distance_lunar < 1
        ORDER BY ca.miss_distance_lunar ASC
        """
    },
    {
        "title": "🪐 Asteroids that came within 0.05 AU",
        "sql": """
        SELECT a.name, ca.close_approach_date, ca.astronomical_AU
        FROM asteroids a
        JOIN close_approach ca ON a.id = ca.neo_reference_id
        WHERE ca.astronomical_AU < 0.05
        ORDER BY ca.astronomical_AU ASC
        """
    },
    {
        "title": "📆 Identify the asteroids that approached Earth on weekend",
        "sql": """
        SELECT * FROM close_approach
        WHERE DAYOFWEEK(close_approach_date) IN (1, 7)
        """
    },
    {
        "title": "🌑 Find asteroids that are unusually dim (magnitude < 20)",
        "sql": """
        SELECT name, magnitude
        FROM asteroids
        WHERE magnitude < 20
        """
    },
    {
        "title": "⚠️ List asteroids with velocity > 2× average velocity",
        "sql": """
        SELECT *
        FROM close_approach
        WHERE relative_velocity_kmph > (
            SELECT 2 * AVG(relative_velocity_kmph) FROM close_approach
        )
        """
    },
    {
        "title": "📍 Find the average miss distance in km for each asteroid",
        "sql": """
        SELECT neo_reference_id,
               COUNT(*) AS approach_count,
               AVG(miss_distance_km) AS avg_miss_distance_km
        FROM close_approach
        GROUP BY neo_reference_id
        """
    },

    {
         "title": "🔎 Asteroids with large diameter range (> 0.3 km)",
         "sql": """
         SELECT id, name, estimated_diameter_min, estimated_diameter_max,
           (estimated_diameter_max - estimated_diameter_min) AS diameter_range
         FROM asteroids
         WHERE (estimated_diameter_max - estimated_diameter_min) > 0.3
         ORDER BY diameter_range DESC
        """
    }

  ]


    query_titles = [q["title"] for q in predefined_queries]
    selected_query_title = st.selectbox("📚 Choose a Predefined Query", query_titles)
    selected_sql = next(q["sql"] for q in predefined_queries if q["title"] == selected_query_title)

    st.code(selected_sql, language="sql")

    if st.button("Run SQL Query"):
        try:
            cursor.execute(selected_sql)
            query_result = cursor.fetchall()
            result_df = pd.DataFrame(query_result)
            st.success("✅ Query executed successfully")
            st.dataframe(result_df, use_container_width=True)
            st.download_button("⬇️ Download Result", result_df.to_csv(index=False), file_name="query_result.csv")
        except Exception as e:
            st.error(f"❌ SQL Error: {e}")

# --- Tab 4: Charts ---
with tab4:
    st.subheader("📊 Visual Analytics")
    if selected_table == "asteroids":
        fig = px.scatter(
            df,
            x="magnitude",
            y="estimated_diameter_max",
            color="is_potentially_hazardous_asteroid",
            title="Magnitude vs Diameter",
            labels={"is_potentially_hazardous_asteroid": "Hazardous?"}
        )
        st.plotly_chart(fig, use_container_width=True)

    elif selected_table == "close_approach":
        st.markdown("### Miss Distance Histogram (km)")
        fig = px.histogram(df, x="miss_distance_km", nbins=40)
        st.plotly_chart(fig, use_container_width=True)


Overwriting app.py


In [44]:
!wget -q -O - ipv4.icanhazip.com

34.106.52.87


In [ ]:
!pip install streamlit

In [45]:
!streamlit run app.py & npx localtunnel --port 8501



⠙⠹⠸⠼
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.106.52.87:8501

⠴⠦⠧⠇⠏your url is: https://gentle-hands-do.loca.lt
  Stopping...
^C
